In [134]:
import numpy as np
import time

In [135]:
m = 2.5
k = 8

In [136]:
def generate_matrix_and_vector(n, dtype=np.float64):
    matrix = np.zeros((n,n), dtype=dtype)

    for i in range(1, n+1):
        for j in range(1, n+1):
            if i == j:
                matrix[i-1, j-1] = k
            elif j > i:
                matrix[i-1, j-1] = (-1) ** j * (m / j)
            elif j == i-1:
                matrix[i-1, j-1] = m/i


    vector = np.array([1 if i % 2 == 0 else -1 for i in range(n)], dtype=dtype)

    return matrix, vector

In [137]:
def difference(v1, v2):
    differences = np.abs(v1 - v2)

    max_difference = np.max(differences)
    return max_difference

In [138]:
def jacobi_iteration_matrix(A):
    D = np.diag(np.diag(A))
    D_inv = np.linalg.inv(D)
    M = np.eye(A.shape[0]) - D_inv @ A
    return M

def spectral_radius(matrix):
    eigenvalues = np.linalg.eigvals(matrix)
    return max(abs(eigenvalues))

In [139]:
# for i in range(3, 200):
#     A, _ = generate_matrix_and_vector(i)
#     M = jacobi_iteration_matrix(A)
#     rho = spectral_radius(M)
#     print(f"n = {i}, rho = {rho}")

In [140]:
def jacobi_method(A, b, rho, type=1, stop=1):
    iterations = 0
    if type == 1:
        x = np.zeros_like(b)
    else:
        x = np.array([100 if i % 2 == 0 else -100 for i in range(b.shape[0])])
        
    D = np.diag(A)
    R = A - np.diagflat(D)
    start_time = time.time()
    while True:
        x_new = (b - R @ x) / D
        iterations += 1
        if stop == 1:
            if np.linalg.norm(x_new - x) < rho:
                break
        else:
            if np.linalg.norm(A @ x_new - b) < rho:
                break
        x = x_new
    end_time = time.time()
            
    return x_new, iterations, end_time - start_time

In [141]:
rhos = [1e-1, 1e-3, 1e-5, 1e-7, 1e-9, 1e-11]

In [142]:
def generate_iterations_table(type, stop):
    table = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    table += "\\begin{tabular}{|c|" + "c|" * len(rhos) + "}\n\\hline\n"
    table += "\\multicolumn{1}{|c|}{} & \\multicolumn{" + str(len(rhos)) + "}{c|}{\\textbf{$\\rho$}} \\\\ \\cline{2-" + str(len(rhos) + 1) + "}\n"
    table += "\\multicolumn{1}{|c|}{\\multirow{-2}{*}{\\textbf{$n$}}} "
    for rho in rhos:
        table += f"& {rho} "
    table += "\\\\ \\hline\n"

    for n in range(5, 351, 20):
        row = f"{n} "
        for rho in rhos:
            A, b = generate_matrix_and_vector(n)
            _, iterations, _ = jacobi_method(A, b, rho=rho, type=type, stop=stop)
            row += f"& {iterations} "
        row += "\\\\ \\hline\n"
        table += row

    table += "\\end{tabular}\n"
    table += "\\captionsetup{justification=centering}\n"
    if type == 1 and stop == 1:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora 0 i pierwszego kryterium stopu}\n\\end{minipage}"
    elif type == 1 and stop == 2:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora 0 i drugiego kryterium stopu}\n\\end{minipage}"
    elif type == 2 and stop == 1:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora 100 i pierwszego kryterium stopu}\n\\end{minipage}"
    else:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora 100 i drugiego kryterium stopu}\n\\end{minipage}"
    return table

In [143]:
def generate_difference_table(type, stop):
    table = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    table += "\\begin{tabular}{|c|" + "c|" * len(rhos) + "}\n\\hline\n"
    table += "\\multicolumn{1}{|c|}{} & \\multicolumn{" + str(len(rhos)) + "}{c|}{\\textbf{$\\rho$}} \\\\ \\cline{2-" + str(len(rhos) + 1) + "}\n"
    table += "\\multicolumn{1}{|c|}{\\multirow{-2}{*}{\\textbf{$n$}}} "
    for rho in rhos:
        table += f"& {rho} "
    table += "\\\\ \\hline\n"

    for n in range(5, 351, 20):
        row = f"{n} "
        for rho in rhos:
            A, b = generate_matrix_and_vector(n)
            x, _, _ = jacobi_method(A, b, rho=rho, type=type, stop=stop)
            diff = difference(b, A @ x)
            row += f"& {diff:.8f} "
        row += "\\\\ \\hline\n"
        table += row

    table += "\\end{tabular}\n"
    table += "\\captionsetup{justification=centering}\n"
    if type == 1 and stop == 1:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora 0 i pierwszego kryterium stopu}\n\\end{minipage}"
    elif type == 1 and stop == 2:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora 0 i drugiego kryterium stopu}\n\\end{minipage}"
    elif type == 2 and stop == 1:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora 100 i pierwszego kryterium stopu}\n\\end{minipage}"
    else:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora 100 i drugiego kryterium stopu}\n\\end{minipage}"
    return table

In [144]:
def generate_execution_time_table(type, stop):
    table = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    table += "\\begin{tabular}{|c|" + "c|" * len(rhos) + "}\n\\hline\n"
    table += "\\multicolumn{1}{|c|}{} & \\multicolumn{" + str(len(rhos)) + "}{c|}{\\textbf{$\\rho$}} \\\\ \\cline{2-" + str(len(rhos) + 1) + "}\n"
    table += "\\multicolumn{1}{|c|}{\\multirow{-2}{*}{\\textbf{$n$}}} "
    for rho in rhos:
        table += f"& {rho} "
    table += "\\\\ \\hline\n"

    for n in range(5, 351, 20):
        row = f"{n} "
        for rho in rhos:
            A, b = generate_matrix_and_vector(n)
            _, _, exec_time = jacobi_method(A, b, rho=rho, type=type, stop=stop)
            row += f"& {exec_time:.8f} "
        row += "\\\\ \\hline\n"
        table += row

    table += "\\end{tabular}\n"
    table += "\\captionsetup{justification=centering}\n"
    if type == 1 and stop == 1:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora 0 i pierwszego kryterium stopu}\n\\end{minipage}"
    elif type == 1 and stop == 2:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora 0 i drugiego kryterium stopu}\n\\end{minipage}"
    elif type == 2 and stop == 1:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora 100 i pierwszego kryterium stopu}\n\\end{minipage}"
    else:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora 100 i drugiego kryterium stopu}\n\\end{minipage}"
    return table

In [145]:
print(generate_iterations_table(1, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 2 & 4 & 7 & 9 & 12 & 14 \\ \hline
25 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
45 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
65 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
85 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
105 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
125 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
145 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
165 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
185 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
205 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
225 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
245 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
265 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
285 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
305 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
325 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
345 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
\end{tabula

In [146]:
print(generate_difference_table(1, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.03203668 & 0.00078881 & 0.00000267 & 0.00000007 & 0.00000000 & 0.00000000 \\ \hline
25 & 0.00581458 & 0.00012548 & 0.00000242 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
45 & 0.00640438 & 0.00014263 & 0.00000235 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
65 & 0.00744797 & 0.00015333 & 0.00000230 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
85 & 0.00822080 & 0.00016114 & 0.00000228 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.00883454 & 0.00016729 & 0.00000230 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
125 & 0.00934353 & 0.00017237 & 0.00000233 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
145 & 0.00977835 & 0.00017669 & 0.00000234 & 0.00000008 & 0.00000000 & 0.00000000 \\ \h

In [147]:
print(generate_execution_time_table(1, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00099850 \\ \hline
25 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00099969 & 0.00000000 \\ \hline
45 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 \\ \hline
65 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00099897 \\ \hline
85 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.00100231 & 0.00000000 & 0.00000000 & 0.00099969 & 0.00100207 & 0.00099969 \\ \hline
125 & 0.00000000 & 0.00000000 & 0.00099897 & 0.00100112 & 0.00099921 & 0.00099921 \\ \hline
145 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00098324 & 0.00100160 \\ \h

In [148]:
print(generate_iterations_table(1, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 2 & 4 & 7 & 9 & 12 & 14 \\ \hline
25 & 2 & 5 & 7 & 9 & 12 & 14 \\ \hline
45 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
65 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
85 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
105 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
125 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
145 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
165 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
185 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
205 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
225 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
245 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
265 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
285 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
305 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
325 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
345 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
\end{tabula

In [149]:
print(generate_difference_table(1, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.03203668 & 0.00078881 & 0.00000267 & 0.00000007 & 0.00000000 & 0.00000000 \\ \hline
25 & 0.06326623 & 0.00012548 & 0.00000242 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
45 & 0.00640438 & 0.00014263 & 0.00000235 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
65 & 0.00744797 & 0.00015333 & 0.00000230 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
85 & 0.00822080 & 0.00016114 & 0.00000228 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.00883454 & 0.00016729 & 0.00000230 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
125 & 0.00934353 & 0.00017237 & 0.00000233 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
145 & 0.00977835 & 0.00017669 & 0.00000234 & 0.00000008 & 0.00000000 & 0.00000000 \\ \h

In [150]:
print(generate_execution_time_table(1, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 \\ \hline
25 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00099874 \\ \hline
45 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 \\ \hline
65 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00100064 & 0.00000000 & 0.00000000 \\ \hline
85 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00100017 & 0.00000000 \\ \hline
105 & 0.00100040 & 0.00099993 & 0.00099969 & 0.00199962 & 0.00152159 & 0.00300002 \\ \hline
125 & 0.00000000 & 0.00000000 & 0.00100136 & 0.00099921 & 0.00099874 & 0.00200081 \\ \hline
145 & 0.00000000 & 0.00100422 & 0.00199986 & 0.00099921 & 0.00200009 & 0.00199986 \\ \h

In [151]:
print(generate_iterations_table(2, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 5 & 8 & 10 & 13 & 15 & 18 \\ \hline
25 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
45 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
65 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
85 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
105 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
125 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
145 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
165 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
185 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
205 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
225 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
245 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
265 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
285 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
305 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
325 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
345 & 6 & 8 & 10 

In [152]:
print(generate_difference_table(2, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.06864039 & 0.00033275 & 0.00000748 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
25 & 0.01928879 & 0.00032974 & 0.00000689 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
45 & 0.02071558 & 0.00032277 & 0.00000660 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
65 & 0.02157283 & 0.00031797 & 0.00000642 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
85 & 0.02218599 & 0.00031434 & 0.00000628 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.02266334 & 0.00031142 & 0.00000617 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
125 & 0.02305415 & 0.00030898 & 0.00000609 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
145 & 0.02338499 & 0.00030688 & 0.00000601 & 0.00000003 & 0.00000000 & 0.00000000 \\ \h

In [153]:
print(generate_execution_time_table(2, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00100088 & 0.00000000 & 0.00000000 \\ \hline
25 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 \\ \hline
45 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 \\ \hline
65 & 0.00000000 & 0.00000000 & 0.00099850 & 0.00000000 & 0.00000000 & 0.00000000 \\ \hline
85 & 0.00099945 & 0.00000000 & 0.00000000 & 0.00100183 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.00099993 & 0.00099993 & 0.00099993 & 0.00100160 & 0.00199962 & 0.00199771 \\ \hline
125 & 0.00099850 & 0.00099826 & 0.00200081 & 0.00100112 & 0.00099993 & 0.00151300 \\ \hline
145 & 0.00100279 & 0.00100303 & 0.00100064 & 0.00200200 & 0.00100160 & 0.00199962 \\ \h

In [154]:
print(generate_iterations_table(2, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
25 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
45 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
65 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
85 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
105 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
125 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
145 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
165 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
185 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
205 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
225 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
245 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
265 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
285 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
305 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
325 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
345 & 6 & 8 & 10 

In [155]:
print(generate_difference_table(2, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.01509448 & 0.00033275 & 0.00000748 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
25 & 0.01928879 & 0.00032974 & 0.00000689 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
45 & 0.02071558 & 0.00032277 & 0.00000660 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
65 & 0.02157283 & 0.00031797 & 0.00000642 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
85 & 0.02218599 & 0.00031434 & 0.00000628 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.02266334 & 0.00031142 & 0.00000617 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
125 & 0.02305415 & 0.00030898 & 0.00000609 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
145 & 0.02338499 & 0.00030688 & 0.00000601 & 0.00000003 & 0.00000000 & 0.00000000 \\ \h

In [156]:
print(generate_execution_time_table(2, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00000000 & 0.00000000 & 0.00099969 & 0.00000000 & 0.00000000 & 0.00000000 \\ \hline
25 & 0.00000000 & 0.00099969 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 \\ \hline
45 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 \\ \hline
65 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 \\ \hline
85 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00000000 & 0.00099921 \\ \hline
105 & 0.00099945 & 0.00100207 & 0.00200009 & 0.00200009 & 0.00300050 & 0.00300121 \\ \hline
125 & 0.00099969 & 0.00100136 & 0.00200105 & 0.00200009 & 0.00299978 & 0.00300097 \\ \hline
145 & 0.00099993 & 0.00100040 & 0.00200009 & 0.00200200 & 0.00200129 & 0.00300241 \\ \h

In [159]:
def generate_latex_table_radius():
    n_values = range(5, 351, 20)
    table_rows = []

    for n in n_values:
        A, _ = generate_matrix_and_vector(n)
        M = jacobi_iteration_matrix(A)
        radius = spectral_radius(M)
        table_rows.append((n, radius))

    latex_code = "\\begin{table}[h!]\n\\centering\n\\begin{tabular}{|c|c|}\n\\hline\nn & Promień spektralny \\\\ \\hline\n"

    for n, radius in table_rows:
        latex_code += f"{n} & {radius:.6f} \\\\ \\hline\n"

    latex_code += "\\end{tabular}\n\\caption{Promienie spektralne dla różnych wartości n}\n\\end{table}"

    return latex_code

In [160]:
print(generate_latex_table_radius())

\begin{table}[h!]
\centering
\begin{tabular}{|c|c|}
\hline
n & Promień spektralny \\ \hline
5 & 0.1562500000 \\ \hline
25 & 0.1562500000 \\ \hline
45 & 0.1562500000 \\ \hline
65 & 0.1562500000 \\ \hline
85 & 0.1562500000 \\ \hline
105 & 0.1562500000 \\ \hline
125 & 0.1562500000 \\ \hline
145 & 0.1562500000 \\ \hline
165 & 0.1562500000 \\ \hline
185 & 0.1562500000 \\ \hline
205 & 0.1562500000 \\ \hline
225 & 0.1562500000 \\ \hline
245 & 0.1562500000 \\ \hline
265 & 0.1562500000 \\ \hline
285 & 0.1562500000 \\ \hline
305 & 0.1562500000 \\ \hline
325 & 0.1562500000 \\ \hline
345 & 0.1562500000 \\ \hline
\end{tabular}
\caption{Promienie spektralne dla różnych wartości n}
\end{table}
